In [3]:
import cv2
import numpy as np
import time
import os, random
import glob

#from PIL import Image
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

from tkinter import *
from tkinter.ttk import Progressbar
from tkinter import ttk

import sys
from config_reader import config_reader
from processing import extract_parts, draw
from model.cmu_model import get_testing_model

# Game Interface

# 1. 시작

In [4]:
def startGameInterface():
    ###### Options ########
    gameTitle = "Welcome to Follow My Lead"
    backgroundImageColor = "black"
    windowAddedHeight = 55
    welcomeText = "Welcome To Follow My Lead"; welcomeTextBgColor="black";  welcomeTextColor = "white"
    buttonText = "Start Game"; buttonBgColor = "black"; buttonTextColor="white";
    progressBarWaitSeconds = 2;

    ###### Create Window ######
    window = Tk()  # Tk 클래스 객체(window) 생성
    window.resizable(False, False)
    window.title(gameTitle)

    ###### Assign Background Image #####
    #photoBg = PhotoImage(file=backgroundImagePath)
    photoBg = PhotoImage(file="a.png")
    window.geometry(str(photoBg.width())+'x'+str(photoBg.height()+windowAddedHeight))
    window.configure(background=backgroundImageColor)
    Label(window, image=photoBg).grid(row=0, column=0, sticky=W)  # , bg="black"

    ###### Assign Welcome Text #####
    Label(window, text=welcomeText, bg=welcomeTextBgColor, fg=welcomeTextColor, font="none 12 bold").grid(row=1, column=0, sticky=W)

    ###### Button ############
    def clicked():
        oldTime = time.time(); finishTime = oldTime + progressBarWaitSeconds
        while (time.time() < finishTime):
            bar['value'] = 100 - int(100 *(finishTime-time.time())/progressBarWaitSeconds)
            window.update_idletasks()
        window.destroy()
        global openGame; openGame=1
    btn = Button(window, text=buttonText,  bg=buttonBgColor, fg=buttonTextColor, font=("Arial Bold", 10), command=clicked)  #, padx=5, pady=5
    btn.grid(column=0, row=0)

    ###### Progress Bar ######
    style = ttk.Style()
    style.theme_use('default')
    style.configure("black.Horizontal.TProgressbar", background='black')
    bar = Progressbar(window, length=400, style='black.Horizontal.TProgressbar')
    bar['value'] = 0
    bar.grid(column=0, row=2)

    #### Start Loop ######
    window.mainloop()

# 2. 메뉴선택
(지금은 아무 이름 버튼 누르면 다음으로 넘어갑니다.) 

In [5]:
def memberchoice():
    ###### Options ########
    windowAddedHeight = 55
    progressBarWaitSeconds = 2;
    bgcolor = "black"; fgcolor = "white"
    ###### Create Window ######
    window = Tk()  # Tk 클래스 객체(window) 생성
    window.resizable(False, False)
    window.title("Member Choice")

    ###### Assign Background Image #####
    photoBg = PhotoImage(file="bts.png")
    Label(window, image=photoBg).pack()

    ###### Button ############
    def clicked():
        window.destroy()
        global openGame; openGame=2
        
    ###### member Button ######     
    btn1 = Button(window, text='뷔',  bg=bgcolor, fg=fgcolor, font=("Arial Bold", 10), width=8, command=clicked).place(x=40, y=300)
    btn2 = Button(window, text='슈가',  bg=bgcolor, fg=fgcolor, font=("Arial Bold", 10), width=8, command=clicked).place(x=120, y=300)
    btn3 = Button(window, text='진',  bg=bgcolor, fg=fgcolor, font=("Arial Bold", 10), width=8, command=clicked).place(x=200, y=300)
    btn4 = Button(window, text='정국',  bg=bgcolor, fg=fgcolor, font=("Arial Bold", 10), width=8, command=clicked).place(x=280, y=300)
    btn5 = Button(window, text='랩몬',  bg=bgcolor, fg=fgcolor, font=("Arial Bold", 10), width=8, command=clicked).place(x=360, y=300)
    btn6 = Button(window, text='지민',  bg=bgcolor, fg=fgcolor, font=("Arial Bold", 10), width=8, command=clicked).place(x=440, y=300)
    btn7 = Button(window, text='제이홉',  bg=bgcolor, fg=fgcolor, font=("Arial Bold", 10), width=8, command=clicked).place(x=520, y=300)
    
    #### Start Loop ######
    window.mainloop()

# 3. 준비

In [6]:
def makeSureBackGroundClearConfirm():

    ######## Options #########
    fontColor = (255, 255, 0); fontScale = 1;  lineType = 2; font = cv2.FONT_HERSHEY_SIMPLEX
    textPositionOriginal = (10, 100)
    welcomeText = 'Ready and Press Enterkey !!!!!!'
    betweenLineSpace = 100

    welcomeTextArray = welcomeText.split('/n')
    
    videoCam = cv2.VideoCapture(0) # index : 카메라 장치 번호 (0 부터 차례로 증가)
    
    # Check web cam is working
    if videoCam.isOpened():       # 캡쳐 객체 연결 확인
        # ret : 프레임 읽기 성공 또는 실패 여부, True / False
        # originalVid : 프레임 이미지, Numpy 배열 또는 None
        ret, originalVid = videoCam.read()     # 다음 프레임 읽기
    else:
        ret = False
        
    # Original Video
    while ret:  # 프레임 읽기가 성공 했다면~
        ret, originalVid = videoCam.read()
        #### Write Text ####
        textPosition = textPositionOriginal
        for i in range(len(welcomeTextArray)):
            cv2.putText(originalVid, welcomeTextArray[i], textPosition, font, fontScale, fontColor, lineType)
            textPositionList = list(textPosition)     #Had to do this to change element in tuple
            textPositionList[1] += betweenLineSpace;
            textPosition = tuple(textPositionList)
        cv2.imshow("Original", originalVid)
        inputKey = cv2.waitKey(1)  # 1ms 지연(40fps로 가정) 
        if inputKey == 13:
            global openGame; openGame=3
            break;
        elif inputKey == 27:
            break;
    cv2.destroyAllWindows()
    videoCam.release()

# Main

# 4. 동작 매칭 게임

In [7]:
def crop(image, w, f):
    return image[:, int(w * f): int(w * (1 - f))]

def main():
    #### load train model ####
    model = load_model('vgg_19.h5')
    img_dim = 128
    class_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
    
    #### load openpose model ####
    posemodel = get_testing_model()
    posemodel.load_weights('model/keras/model.h5')
    params, model_params = config_reader()
    
    #### option #### 
    factor = 0.1
    scale_search = [0.22, 0.25, .5, 1, 1.5, 2]  
    process_speed = 1
    scale_search = scale_search[0:process_speed]
    params['scale_search'] = scale_search
    resize_fac = 5
    
    cap = cv2.VideoCapture(0)
    
    number = [1, 2, 1, 3, 4, 5, 6, 1, 2, 3, 4, 7, 8, 9, 10, 11]
    name_list = np.array(sorted(glob.glob("show_img/*")))
    i = 0
    
    while True:
        if i < len(number):
            name = name_list[number[i]-1]
            icon = cv2.imread('{}'.format(name))
            icon = cv2.resize(icon, (400, 380))
            cv2.imshow('Pose', icon)        
            order = number[i]

            ret, frame = cap.read()
            width = frame.shape[1]
            height = frame.shape[0]
            # webcam space
            cv2.rectangle(frame, (100, 100), (500, 500), (255, 255, 255), 2)
            roi = frame[100:500, 100:500]
            img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_dim, img_dim))

            #################### webcam 포즈 예측 #################### 
            img = image.img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img = img.astype('float32')/255
            pred = np.argmax(model.predict(img)) 
            cv2.putText(frame, class_labels[pred], (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 2) 
            
            #################### webcam openpose 예측 #################### 
            frame = crop(frame, width, factor)
            input_image = cv2.resize(frame, (0, 0), fx=1/resize_fac, fy=1/resize_fac, interpolation=cv2.INTER_CUBIC)
            input_image = cv2.cvtColor(input_image, cv2.COLOR_RGB2BGR)
            # generate image with body parts
            body_parts, all_peaks, subset, candidate = extract_parts(input_image, params, posemodel, model_params)
            frame = draw(frame, all_peaks, subset, candidate, resize_fac=resize_fac)
            
            frame = cv2.flip(frame, 1)
            cv2.imshow('Webcam', frame)
            
            if str(class_labels[pred]) == str(order):  
                i += 1
                print(i)
                
            else:
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                ret, frame = cap.read()    
        else:
            break
                   
    cap.release()
    cv2.destroyAllWindows()              

In [8]:
openGame = 0;   # If you wanna skip interface, assign this to 2

startGameInterface()

if(openGame==1):
    memberchoice()

if(openGame==2):
    makeSureBackGroundClearConfirm()

if(openGame==3):
    if __name__=="__main__":
        main()

1
